In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "LeNet5_ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[3] + "_LRdown_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[3] +  "_LRdown_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[3] + "_LRdown"

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df[df.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold, setLR):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = setLR
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_17024\2625783892.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 131.19 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #學習率
    lr=0
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #1s
            dataX = dataX1
            dataY = dataY1
            lr = 0.0032
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
            lr = 0.0016
        else:
            #5s
            dataX = dataX5
            dataY = dataY5
            lr = 0.0008

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(trainFold=fold, setLR=lr)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 3.440767526626587 Train_acc: 5.562500%
Train Epoch: 2/60 Train_Loss: 3.150691509246826 Train_acc: 9.625000%
Train Epoch: 3/60 Train_Loss: 3.030142307281494 Train_acc: 13.462501%
Train Epoch: 4/60 Train_Loss: 2.782986640930176 Train_acc: 17.087502%
Train Epoch: 5/60 Train_Loss: 2.858858585357666 Train_acc: 18.087502%
Train Epoch: 6/60 Train_Loss: 3.0513124465942383 Train_acc: 22.437502%
Train Epoch: 7/60 Train_Loss: 2.3083465099334717 Train_acc: 23.937502%
Train Epoch: 8/60 Train_Loss: 2.735499382019043 Train_acc: 25.650002%
Train Epoch: 9/60 Train_Loss: 2.5738327503204346 Train_acc: 28.187502%
Train Epoch: 10/60 Train_Loss: 2.5980591773986816 Train_acc: 28.812502%
Train Epoch: 11/60 Train_Loss: 2.36235523

Train Epoch: 47/60 Train_Loss: 0.36577555537223816 Train_acc: 88.250000%
Train Epoch: 48/60 Train_Loss: 0.5211359262466431 Train_acc: 88.354172%
Train Epoch: 49/60 Train_Loss: 0.49409595131874084 Train_acc: 88.729172%
Train Epoch: 50/60 Train_Loss: 0.36941036581993103 Train_acc: 88.812500%
Train Epoch: 51/60 Train_Loss: 0.3673219084739685 Train_acc: 88.937500%
Train Epoch: 52/60 Train_Loss: 0.30079999566078186 Train_acc: 89.145836%
Train Epoch: 53/60 Train_Loss: 0.592728316783905 Train_acc: 88.958336%
Train Epoch: 54/60 Train_Loss: 0.44216832518577576 Train_acc: 89.166672%
Train Epoch: 55/60 Train_Loss: 0.5699631571769714 Train_acc: 89.375000%
Train Epoch: 56/60 Train_Loss: 0.46738865971565247 Train_acc: 89.062500%
Train Epoch: 57/60 Train_Loss: 0.3401443064212799 Train_acc: 89.145836%
Train Epoch: 58/60 Train_Loss: 0.44488781690597534 Train_acc: 88.770836%
Train Epoch: 59/60 Train_Loss: 0.4677598178386688 Train_acc: 89.020836%
Train Epoch: 60/60 Train_Loss: 0.28987109661102295 Train_a

Train Epoch: 30/60 Train_Loss: 1.6165939569473267 Train_acc: 56.612503%
Train Epoch: 31/60 Train_Loss: 1.4219324588775635 Train_acc: 60.450005%
Train Epoch: 32/60 Train_Loss: 1.429720163345337 Train_acc: 61.825005%
Train Epoch: 33/60 Train_Loss: 0.8877542018890381 Train_acc: 62.087502%
Train Epoch: 34/60 Train_Loss: 1.4606837034225464 Train_acc: 63.000004%
Train Epoch: 35/60 Train_Loss: 1.593260645866394 Train_acc: 63.487503%
Train Epoch: 36/60 Train_Loss: 1.2937045097351074 Train_acc: 62.612503%
Train Epoch: 37/60 Train_Loss: 1.187506079673767 Train_acc: 63.275002%
Train Epoch: 38/60 Train_Loss: 1.4039056301116943 Train_acc: 64.162506%
Train Epoch: 39/60 Train_Loss: 1.1261560916900635 Train_acc: 64.350006%
Train Epoch: 40/60 Train_Loss: 1.080999493598938 Train_acc: 64.475006%
Train Epoch: 41/60 Train_Loss: 1.4015263319015503 Train_acc: 64.862503%
Train Epoch: 42/60 Train_Loss: 1.4416905641555786 Train_acc: 64.862503%
Train Epoch: 43/60 Train_Loss: 1.0759493112564087 Train_acc: 65.1625

Train Epoch: 14/60 Train_Loss: 0.144361674785614 Train_acc: 95.000000%
Train Epoch: 15/60 Train_Loss: 0.20013569295406342 Train_acc: 94.000000%
Train Epoch: 16/60 Train_Loss: 0.2343738079071045 Train_acc: 95.937500%
Train Epoch: 17/60 Train_Loss: 0.21033799648284912 Train_acc: 96.750000%
Train Epoch: 18/60 Train_Loss: 0.16356319189071655 Train_acc: 96.437500%
Train Epoch: 19/60 Train_Loss: 0.13450513780117035 Train_acc: 96.437500%
Train Epoch: 20/60 Train_Loss: 0.18309739232063293 Train_acc: 97.062500%
Train Epoch: 21/60 Train_Loss: 0.12335116416215897 Train_acc: 96.937500%
Train Epoch: 22/60 Train_Loss: 0.16444413363933563 Train_acc: 96.562500%
Train Epoch: 23/60 Train_Loss: 0.07144322991371155 Train_acc: 96.437500%
Train Epoch: 24/60 Train_Loss: 0.1128680631518364 Train_acc: 97.375000%
Train Epoch: 25/60 Train_Loss: 0.0992099940776825 Train_acc: 97.500000%
Train Epoch: 26/60 Train_Loss: 0.12420274317264557 Train_acc: 97.687500%
Train Epoch: 27/60 Train_Loss: 0.17574305832386017 Train

Train Epoch: 1/60 Train_Loss: 1.3673465251922607 Train_acc: 62.250000%
Train Epoch: 2/60 Train_Loss: 0.9336076378822327 Train_acc: 67.125000%
Train Epoch: 3/60 Train_Loss: 0.9129132628440857 Train_acc: 69.625000%
Train Epoch: 4/60 Train_Loss: 0.803648829460144 Train_acc: 70.437500%
Train Epoch: 5/60 Train_Loss: 0.8630319237709045 Train_acc: 69.375000%
Train Epoch: 6/60 Train_Loss: 0.8500897288322449 Train_acc: 71.958336%
Train Epoch: 7/60 Train_Loss: 0.9567193984985352 Train_acc: 71.791672%
Train Epoch: 8/60 Train_Loss: 0.6245545148849487 Train_acc: 73.625000%
Train Epoch: 9/60 Train_Loss: 0.9624398946762085 Train_acc: 72.562500%
Train Epoch: 10/60 Train_Loss: 0.8601434826850891 Train_acc: 73.375000%
Train Epoch: 11/60 Train_Loss: 0.659564733505249 Train_acc: 73.729172%
Train Epoch: 12/60 Train_Loss: 0.6413627862930298 Train_acc: 75.583336%
Train Epoch: 13/60 Train_Loss: 0.9567376375198364 Train_acc: 74.270836%
Train Epoch: 14/60 Train_Loss: 0.6695684790611267 Train_acc: 75.854172%
Tra

Train Epoch: 50/60 Train_Loss: 0.06150517240166664 Train_acc: 99.000000%
Train Epoch: 51/60 Train_Loss: 0.07509487867355347 Train_acc: 98.937500%
Train Epoch: 52/60 Train_Loss: 0.02853056602180004 Train_acc: 98.937500%
Train Epoch: 53/60 Train_Loss: 0.0656626969575882 Train_acc: 98.875000%
Train Epoch: 54/60 Train_Loss: 0.07523635029792786 Train_acc: 98.937500%
Train Epoch: 55/60 Train_Loss: 0.06928279250860214 Train_acc: 99.062500%
Train Epoch: 56/60 Train_Loss: 0.03923754394054413 Train_acc: 98.937500%
Train Epoch: 57/60 Train_Loss: 0.06020975112915039 Train_acc: 99.125000%
Train Epoch: 58/60 Train_Loss: 0.039776504039764404 Train_acc: 99.000000%
Train Epoch: 59/60 Train_Loss: 0.08577487617731094 Train_acc: 98.937500%
Train Epoch: 60/60 Train_Loss: 0.0563492476940155 Train_acc: 99.125000%
Test accuracy of the model: 31.650000%
Test accuracy of the model: 44.750000%
Test accuracy of the model: 48.500000%
Training Time: 38.72 seconds
可用的 GPU 数量: 1
--------------------------------------

Train Epoch: 34/60 Train_Loss: 0.5638613700866699 Train_acc: 86.104172%
Train Epoch: 35/60 Train_Loss: 0.7446080446243286 Train_acc: 86.041672%
Train Epoch: 36/60 Train_Loss: 0.3322773277759552 Train_acc: 86.104172%
Train Epoch: 37/60 Train_Loss: 0.5769335627555847 Train_acc: 85.666672%
Train Epoch: 38/60 Train_Loss: 0.3898769021034241 Train_acc: 85.791672%
Train Epoch: 39/60 Train_Loss: 0.3867330253124237 Train_acc: 86.437500%
Train Epoch: 40/60 Train_Loss: 0.6717124581336975 Train_acc: 85.729172%
Train Epoch: 41/60 Train_Loss: 0.4636710584163666 Train_acc: 86.395836%
Train Epoch: 42/60 Train_Loss: 0.4832773208618164 Train_acc: 86.187500%
Train Epoch: 43/60 Train_Loss: 0.5471972227096558 Train_acc: 87.020836%
Train Epoch: 44/60 Train_Loss: 0.6455137729644775 Train_acc: 85.937500%
Train Epoch: 45/60 Train_Loss: 0.2413204163312912 Train_acc: 87.041672%
Train Epoch: 46/60 Train_Loss: 0.5567644238471985 Train_acc: 87.875000%
Train Epoch: 47/60 Train_Loss: 0.598583996295929 Train_acc: 88.1

Train Epoch: 17/60 Train_Loss: 1.8135581016540527 Train_acc: 45.437504%
Train Epoch: 18/60 Train_Loss: 1.5740009546279907 Train_acc: 46.200001%
Train Epoch: 19/60 Train_Loss: 1.5505282878875732 Train_acc: 44.850002%
Train Epoch: 20/60 Train_Loss: 1.5877048969268799 Train_acc: 47.187504%
Train Epoch: 21/60 Train_Loss: 1.754308819770813 Train_acc: 48.262501%
Train Epoch: 22/60 Train_Loss: 2.102768898010254 Train_acc: 47.912502%
Train Epoch: 23/60 Train_Loss: 1.5834357738494873 Train_acc: 48.500004%
Train Epoch: 24/60 Train_Loss: 1.8854529857635498 Train_acc: 49.237503%
Train Epoch: 25/60 Train_Loss: 1.6913421154022217 Train_acc: 50.512501%
Train Epoch: 26/60 Train_Loss: 1.711134433746338 Train_acc: 51.137501%
Train Epoch: 27/60 Train_Loss: 1.6217379570007324 Train_acc: 51.250004%
Train Epoch: 28/60 Train_Loss: 1.7274008989334106 Train_acc: 52.250004%
Train Epoch: 29/60 Train_Loss: 1.6159775257110596 Train_acc: 53.487503%
Train Epoch: 30/60 Train_Loss: 1.5866984128952026 Train_acc: 53.600

Train Epoch: 1/60 Train_Loss: 0.5085963606834412 Train_acc: 80.062500%
Train Epoch: 2/60 Train_Loss: 0.22186040878295898 Train_acc: 87.875000%
Train Epoch: 3/60 Train_Loss: 0.33548417687416077 Train_acc: 89.625000%
Train Epoch: 4/60 Train_Loss: 0.4325416088104248 Train_acc: 90.250000%
Train Epoch: 5/60 Train_Loss: 0.27245911955833435 Train_acc: 91.000000%
Train Epoch: 6/60 Train_Loss: 0.2932160198688507 Train_acc: 90.562500%
Train Epoch: 7/60 Train_Loss: 0.34711435437202454 Train_acc: 91.687500%
Train Epoch: 8/60 Train_Loss: 0.3276616632938385 Train_acc: 92.312500%
Train Epoch: 9/60 Train_Loss: 0.17930303514003754 Train_acc: 93.437500%
Train Epoch: 10/60 Train_Loss: 0.17687858641147614 Train_acc: 93.562500%
Train Epoch: 11/60 Train_Loss: 0.22331537306308746 Train_acc: 92.937500%
Train Epoch: 12/60 Train_Loss: 0.4374362826347351 Train_acc: 92.250000%
Train Epoch: 13/60 Train_Loss: 0.35717713832855225 Train_acc: 91.250000%
Train Epoch: 14/60 Train_Loss: 0.17860816419124603 Train_acc: 93.

In [9]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()